In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
import sklearn.datasets
import sklearn.ensemble
import lime
import lime.lime_tabular
from __future__ import print_function

In [2]:
# Part 1: Develop a regression model to predict Yield_CT considering the columns except (Yield_NT, NT_effect). Perform some cleaning process if needed.   (30 points)

# Consider 5 instances and apply LIME to explain the model's decision 

 

# Part 2: Develop a regression model to predict Yield_NT considering the columns except (Yield_CT, NT_effect). Perform some cleaning process if needed.   (30 points)

# Consider 5 instances and apply LIME to explain the model's decision 

In [3]:
#df.to_csv('preprocessed_Yield_NT.csv', encoding='utf-8', index=False)
df = pd.read_csv('./preprocessed_Yield_NT.csv', low_memory=False) 
#df.info()

In [4]:
feat_cols = list(df.columns)
label_col = "Yield_NT"
feat_cols.remove(label_col)
X = df.drop([label_col], axis=1)
y = df[label_col]

feat_cols = list(X.columns)
from sklearn.preprocessing import LabelEncoder
categorical_index=[]
feat_cols = list(X.columns)
categorical_names = {}

categorical_features = ["Crop",'ST']
categorical_index = [feat_cols.index(i) for i in categorical_features]
print(categorical_index)
# for i in categorical_features:
#     z=feat_cols.index(i)
#     print(z)
#     categorical_index.append(z)
df.info()
print("feat_col_len"+str(len(feat_cols)))
def Label_encoding(X, name,index):
    le = LabelEncoder()
    label = le.fit_transform(X[name])
   # df.drop(name, axis=1, inplace=True)
    X[name] = label
    le_value_mapping = dict(zip(le.transform(le.classes_), le.classes_))
    categorical_names[index] = le_value_mapping
    #categorical_names[index] = le.classes_
    print(name)
    print(le.classes_)

for i in categorical_features:
    z=feat_cols.index(i)
    Label_encoding(X,i,z)
print("after encoding \n")
X.info()


categorical_names
# feat_cols = list(df.columns)
# feat_cols.remove(label_col)


[3, 14]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3295 entries, 0 to 3294
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Yield_NT              3295 non-null   float64
 1   Yield_change          3295 non-null   float64
 2   Latitude              3295 non-null   float64
 3   Longitude             3295 non-null   float64
 4   Crop                  3295 non-null   object 
 5   Years_NT              3295 non-null   int64  
 6   Crop_rotation_NT      3295 non-null   int64  
 7   Soil_cover_NT         3295 non-null   float64
 8   Weed_pest_control_NT  3295 non-null   float64
 9   P                     3295 non-null   float64
 10  E                     3295 non-null   float64
 11  PB                    3295 non-null   float64
 12  Tave                  3295 non-null   float64
 13  Tmax                  3295 non-null   float64
 14  Tmin                  3295 non-null   float64
 15  ST           

{3: {0: 'barley.spring',
  1: 'barley.winter',
  2: 'cotton',
  3: 'maize',
  4: 'rice',
  5: 'sorghum',
  6: 'soybean',
  7: 'sunflower',
  8: 'wheat.spring',
  9: 'wheat.winter'},
 14: {0: 'Clay',
  1: 'Clay loam',
  2: 'Loam',
  3: 'Sandy clay',
  4: 'Sandy clay loam',
  5: 'Sandy loam',
  6: 'Silt loam'}}

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

#del X
#del y

In [6]:
#Build the model with Random Forest Regressor :
from sklearn.ensemble import RandomForestRegressor
import pickle
model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
model.fit(X_train, y_train)

RandomForestRegressor(max_depth=6, n_estimators=10, random_state=0)

In [7]:
# # save the model to disk
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [8]:
# load the model from disk
filename = 'finalized_model.sav'
model = pickle.load(open(filename, 'rb'))
result = model.score(X_test, y_test)
print(result)

0.5115736735646944


In [9]:
y_pred = model.predict(X_test)

In [10]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)**(0.5)
mse

2611.446860508777

In [11]:
print('Random Forest MSError', np.mean((model.predict(X_test) - y_test) ** 2))

Random Forest MSError 6819654.705261147


In [12]:
#from sklearn.metrics import r2_score
model.score(X_test, y_test)

0.5115736735646944

In [13]:
print('MSError when predicting the mean', np.mean((y_train.mean() - y_test) ** 2))

MSError when predicting the mean 14029159.57149537


In [14]:
from mlsocket import MLSocket
import socket
import numpy as np
HOST = socket.gethostname()
PORT = 65432
data1 = np.array([1, 2, 3, 4])
with MLSocket() as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, address = s.accept()

    with conn:
        received_data = conn.recv(1024) # This will block until it receives all the data send by the client, with the step size of 1024 bytes.
        #model = conn.recv(1024) # This will also block until it receives all the data.
        #clf = conn.recv(1024) # Same
        print(received_data)
        received_data=received_data.reshape(1,-1)
        y_pred = model.predict(received_data)
        print(y_pred)
        conn.send(y_pred)
        #conn.send("y_pred")

[ 1.602000e-01  1.952000e+01 -9.883000e+01  9.000000e+00  6.000000e+00
  1.000000e+00  0.000000e+00  1.000000e+00  4.012000e+02  6.890000e+02
 -2.878000e+02  1.398571e+01  2.452044e+01  2.949670e+00  2.000000e+00]
[3429.11196001]


C:\Users\Masrufa\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
